In [1]:
import pandas as pd
import numpy as np
import duckdb
import time

In [3]:
def connect_db(path: str):
    '''Runs duckdb.connect() function on database path with timing. Returns a 
duckdb.DuckDBPyConnection object'''
    
    st = time.time()

    print('Connecting to database...')
    con = duckdb.connect(path)

    et = time.time()
    elapsed_time = et - st
    print(f'Connection established! Execution time: {elapsed_time} seconds')
    return con

In [4]:
con = connect_db('/mnt/c/Users/ryanf/Desktop/validprot_testing')

Connecting to database...
Connection established! Execution time: 0.29888248443603516 seconds


In [7]:
duckdb.__version__

'0.6.1'

## Taxa pairs

In [4]:
# Builds ValidProt taxa pair table using only paired taxa from learn2therm
taxa_pairs_cmd = """CREATE OR REPLACE TABLE vp_taxa_pairs AS SELECT * FROM taxa_pairs WHERE is_pair = True"""
con.execute(taxa_pairs_cmd)

In [5]:
print(con.execute("""SELECT COUNT(thermo_index) FROM vp_taxa_pairs""").df())
con.execute("""SELECT * FROM vp_taxa_pairs LIMIT 1""").df()

   count(thermo_index)
0                  199


,thermo_index,meso_index,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,local_E_value,query_align_start,query_align_end,subject_align_end,subject_align_start,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,taxa_pair_index,is_pair
0,14062,12750,0.935336,0.934116,0.934116,0.0,8,1533,1533,8,1526,0.995434,1526,0.995434,1214.0,617768,True


## Taxa

In [6]:
# Commands to identify all taxa that are implicated in learn2therm pairs.
meso_cmd = """SELECT DISTINCT meso_index
FROM taxa_pairs
WHERE is_pair = True"""

thermo_cmd = """SELECT DISTINCT thermo_index
FROM taxa_pairs
WHERE is_pair = True"""

useful_thermo = con.execute(thermo_cmd).df()
useful_meso = con.execute(meso_cmd).df()

# Generates tuple object containing all relevant taxa
useful_taxa = tuple([i for i in useful_meso['meso_index']] + [i for i in useful_thermo['thermo_index']])

# Builds ValidProt taxa table using only paired taxa from learn2therm.
taxa_cmd = f"""CREATE OR REPLACE TABLE vp_taxa AS SELECT * FROM taxa WHERE taxa_index IN {useful_taxa}"""
con.execute(taxa_cmd)

In [7]:
print(con.execute("""SELECT COUNT(taxa_index) FROM vp_taxa""").df())
con.execute("""SELECT * FROM vp_taxa LIMIT 1""").df()

   count(taxa_index)
0                250


,taxa_index,ncbi_taxid,record_name,filepath,taxonomy,organism,bacdive_id,ogt_scraped_string,seq_16srRNA,len_16s,ogt,thermophile_label
0,31,400768,NZ_JAFBCF010000001,./data/refseq/bacteria/GCF_016907535.1_ASM1690...,Bacteria Actinobacteria Propionibacteriales Pr...,Microlunatus panaciterrae,12627,28,TCAACGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTG...,1519,28.0,False


## ogt and 16S filtered pairs

In [8]:
min_ogt_diff = 20
min_16s = 1300
# Builds ValidProt table containing taxa pairs and their associated optimal growth temperatures (ogt). 
# Excludes short 16S sequences and pairs with ogt spread below cutoff value from function input.
ogt_pairs_cmd = f"""CREATE OR REPLACE TABLE vp_ogt_taxa_pairs AS SELECT vp_taxa_pairs.*,
                taxa_m.ogt AS meso_ogt,
                taxa_t.ogt AS thermo_ogt,
                taxa_t.ogt - taxa_m.ogt AS ogt_diff,
                taxa_m.len_16s AS meso_16s_len,
                taxa_t.len_16s AS thermo_16s_len
                FROM vp_taxa_pairs
                JOIN vp_taxa AS taxa_m ON (vp_taxa_pairs.meso_index = taxa_m.taxa_index)
                JOIN vp_taxa AS taxa_t ON (vp_taxa_pairs.thermo_index = taxa_t.taxa_index)
                WHERE ogt_diff >= {min_ogt_diff}
                AND meso_16s_len >= {min_16s}
                AND thermo_16s_len >= {min_16s}"""
con.execute(ogt_pairs_cmd)

In [9]:
print(con.execute("""SELECT COUNT(meso_index) FROM vp_ogt_taxa_pairs""").df())
con.execute("""SELECT * FROM vp_ogt_taxa_pairs LIMIT 1""").df()

   count(meso_index)
0                253


,thermo_index,meso_index,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,local_E_value,query_align_start,query_align_end,subject_align_end,subject_align_start,...,subject_align_len,subject_align_cov,bit_score,taxa_pair_index,is_pair,meso_ogt,thermo_ogt,ogt_diff,meso_16s_len,thermo_16s_len
0,875,16030,0.917979,0.913782,0.909919,0.0,6,1531,1544,6,...,1539,0.996762,1098.0,32298,True,28.0,50.0,22.0,1544,1531


## Protein pairs

In [10]:
# Builds ValidProt table containing protein pairs 
protein_pair_cmd = """CREATE OR REPLACE TABLE vp_protein_pairs AS SELECT protein_pairs.*,
                otp.local_gap_compressed_percent_id AS local_gap_compressed_percent_id_16s,
                otp.scaled_local_query_percent_id AS scaled_local_query_percent_id_16s,
                otp.scaled_local_symmetric_percent_id AS scaled_local_symmetric_percent_id_16s,
                otp.query_align_cov AS query_align_cov_16s,
                otp.subject_align_cov AS subject_align_cov_16s,
                otp.bit_score AS bit_score_16s,
                otp.meso_ogt AS m_ogt,
                otp.thermo_ogt AS t_ogt,
                otp.ogt_diff AS ogt_difference
                FROM protein_pairs 
                INNER JOIN vp_ogt_taxa_pairs AS otp ON (protein_pairs.taxa_pair_index = otp.taxa_pair_index)"""

con.execute(protein_pair_cmd)

In [11]:
print(con.execute("""SELECT COUNT(meso_index) FROM vp_protein_pairs""").df())
con.execute("""SELECT * FROM vp_protein_pairs LIMIT 1""").df()

   count(meso_index)
0                263


,thermo_protein_index,meso_protein_index,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,local_E_value,query_align_start,query_align_end,subject_align_end,subject_align_start,...,taxa_pair_index,local_gap_compressed_percent_id_16s,scaled_local_query_percent_id_16s,scaled_local_symmetric_percent_id_16s,query_align_cov_16s,subject_align_cov_16s,bit_score_16s,m_ogt,t_ogt,ogt_difference
0,875.2516,16030.1214,0.308725,0.166667,0.154882,0.0185,56,210,250,91,...,32298,0.917979,0.913782,0.909919,0.996734,0.996762,1098.0,28.0,50.0,22.0


In [12]:
# Builds ValidProt table containing proteins that belong to taxa pairs from vp_taxa_pairs.
prot_filt_cmd = """CREATE OR REPLACE TABLE vp_proteins AS SELECT *
FROM proteins
WHERE protein_int_index IN (SELECT DISTINCT meso_protein_int_index FROM vp_protein_pairs) OR
protein_int_index IN (SELECT DISTINCT thermo_protein_int_index FROM vp_protein_pairs)
"""
con.execute(prot_filt_cmd)

In [13]:
print(con.execute("""SELECT COUNT(protein_int_index) FROM vp_proteins""").df())
con.execute("""SELECT * FROM vp_proteins LIMIT 1""").df()

   count(protein_int_index)
0                       264


,taxa_index,protein_index,protein_seq,protein_desc,protein_len,protein_int_index
0,7626,7626.1967,MATLKAQHLAKSYKGRQVVRDVSLSIESGQIVGLLGPNGAGKTTCF...,LPS export ABC transporter ATP-binding protein,241,20546221


## Final table

In [14]:
big_table_cmd = """CREATE OR REPLACE TABLE vp_final AS SELECT vp_protein_pairs.*,
proteins_m.protein_seq AS m_protein_seq,
proteins_t.protein_seq AS t_protein_seq,
proteins_m.protein_desc AS m_protein_desc,
proteins_t.protein_desc AS t_protein_desc,
proteins_m.protein_len AS m_protein_len,
proteins_t.protein_len AS t_protein_len
FROM vp_protein_pairs
JOIN vp_proteins AS proteins_m ON (vp_protein_pairs.meso_protein_int_index = proteins_m.protein_int_index)
JOIN vp_proteins AS proteins_t ON (vp_protein_pairs.thermo_protein_int_index = proteins_t.protein_int_index)"""

con.execute(big_table_cmd)

In [15]:
print(con.execute("""SELECT COUNT(meso_protein_int_index) FROM vp_final""").df())
con.execute("""SELECT * FROM vp_final LIMIT 1""").df()

   count(meso_protein_int_index)
0                            269


,thermo_protein_index,meso_protein_index,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,local_E_value,query_align_start,query_align_end,subject_align_end,subject_align_start,...,bit_score_16s,m_ogt,t_ogt,ogt_difference,m_protein_seq,t_protein_seq,m_protein_desc,t_protein_desc,m_protein_len,t_protein_len
0,7134.186,14485.58,0.384106,0.246809,0.216418,4.740000e-07,30,183,256,89,...,1051.0,27.5,52.5,25.0,MTRPEEPIEEVRVDVESARAANRANWDDRARVHADSTEYDLPGLAA...,MIWRLVRAALEEQVEVAARSVLDVVDAGGGTGNLAVPIARLGHRVT...,methyltransferase domain-containing protein,methyltransferase domain-containing protein,301,235


In [16]:
con.commit()
con.close()

In [18]:
con.execute("""SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE='BASE TABLE'""").df()

,table_name
0,vp_final
1,vp_proteins
2,vp_protein_pairs
3,vp_ogt_taxa_pairs
4,vp_taxa
5,vp_taxa_pairs
6,protein_pairs
7,proteins
8,taxa_pairs
9,taxa


In [2]:
for i in range(0,10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [4]:
import os
os.path.join('..','c0_preprocessing')

'../c0_preprocessing'

In [7]:
import duckdb

con = duckdb.connect('validprot_empty')

In [8]:
con.commit()
con.close()